In [1]:
# Web-learner for pyscipopt
# https://www.youtube.com/watch?v=7SvapoXVYq0&t=1902s
# https://scipopt.github.io/PySCIPOpt/docs/html/classpyscipopt_1_1scip_1_1Model.html#a59ac011e2fcca0dec49e8da06443ba90

# Example from following example
# https://github.com/wujianjack/optimizationmodels/tree/master/gurobi

'''
利用pyscipopt复现了上述案例中的Lagrangian-relaxation
gurobi中有些函数功能在 pyscipopt 中是存在的 (例如 remove constraints)
在复现过程中，采用了一个不经济的方法去更新 UB: 重新 update 变量的 LB、UB 以达到fix-ariable的效果
可以参考使用 delCons的方法，将 LR 模型与 Fixed模型整合在一起(像案例那样去做)
'''

In [3]:
import pyscipopt
from pyscipopt import Model, quicksum, Expr

class LocationTransport:
    def __init__(self):
        # initialize data
        self.buildlimit = 0   # 8 可选上限
        self.ncites = 0       # 30个客户 
        self.supply = []      # 30个仓库
        self.demand = []
        self.shipcost = []

        # initialize parameters
        self.iterlimit = 100
        self.samelimit = 3
        
        self.steplog = []
        self.scalelog = []
        self.xLBlog = []
        self.xUBlog = []

        # variable list, constraint list for models        
        self.vship = []  # var x(i,j)
        self.vbuild = [] # var y(i)
        
        self.vship_ld = []   # var x(i,j) in Lagrangian-dual prob
        self.vbuild_ld = []  # var y(i)
        self.crelax_ld = []  # relaxed constraints
        
    def read(self, filename):
        with open(filename, "r") as data:
            self.buildlimit = int(data.readline())
            self.ncites = int(data.readline())
            
            column = data.readline().split()
            for i in range(self.ncites):
                self.supply.append(float(column[i]))
            
            column = data.readline().split()
            for i in range(self.ncites):
                self.demand.append(float(column[i]))
            
            for i in range(self.ncites):
                column = data.readline().split()
                lshipcost = []
                for j in range(self.ncites):
                    lshipcost.append(float(column[j]))
                self.shipcost.append(lshipcost)

    def build(self):
        self.mtrans = Model("IP-LR")
        self.mtrans.hideOutput(True)
        
        # Define variable
        for i in range(self.ncites):
            shipvar = []
            for j in range(self.ncites):
                shipvar.append(self.mtrans.addVar(vtype='I', lb=0.0, ub=self.demand[j]))
            self.vship.append(shipvar)

        for i in range(self.ncites):
            self.vbuild.append(self.mtrans.addVar(vtype='B', lb=0.0, ub=1.0))
        
        # Define constraints
        for i in range(self.ncites):
            self.mtrans.addCons(quicksum(self.vship[i][j] for j in range(self.ncites)) \
                                  <= self.supply[i] * self.vbuild[i])

        self.mtrans.addCons(quicksum(self.vbuild[i] for i in range(self.ncites)) \
                              <= self.buildlimit)
        
    def build_fixed(self):
        self.mtrans_ld = Model("LD")
        self.mtrans_ld.hideOutput(True)
        
        # Define var
        for i in range(self.ncites):
            shipvar_ld = []
            for j in range(self.ncites):
                shipvar_ld.append(self.mtrans_ld.addVar(name="x_"+str(i)+"_"+str(j), vtype='I', lb=0.0, ub=self.demand[j]))
            self.vship_ld.append(shipvar_ld)
            
        for i in range(self.ncites):
            self.vbuild_ld.append(self.mtrans_ld.addVar(name="y_"+str(i), vtype='B', lb=0.0, ub=1.0))


        # add constraints
        for i in range(self.ncites):
            self.mtrans_ld.addCons(quicksum(self.vship_ld[i][j] for j in range(self.ncites)) \
                                <= self.supply[i] * self.vbuild_ld[i])
        
        self.mtrans_ld.addCons(quicksum(self.vbuild_ld[i] for i in range(self.ncites)) <= self.buildlimit)
        
        # Relaxed constraints >>>
        for j in range(self.ncites):
            self.crelax_ld.append(self.mtrans_ld.addCons(quicksum(self.vship_ld[i][j] \
                               for i in range(self.ncites)) \
                               >= self.demand[j]))
                
        obj_shipcost_fixed = quicksum(self.vship_ld[i][j] * self.shipcost[i][j] \
                                      for i in range(self.ncites) \
                                      for j in range(self.ncites))
                
        self.mtrans_ld.setObjective(obj_shipcost_fixed, sense="minimize")
    
    def update_bound(self, sol):
        for i in range(self.ncites):
            self.mtrans_ld.chgVarLb(self.vbuild_ld[i], sol[i])
            self.mtrans_ld.chgVarUb(self.vbuild_ld[i], sol[i])
        
    def solve(self):
        # 'Lagrangian Relaxation' parameters
        same = 0         # noChangeCnt
        norm = 0.0       # squareSum
        step = 0.0       # step-size
        scale = 1.0      # theta
        xLB = 0.0
        xUB = 0.0
        
        xlambda = [0.0] * self.ncites   # lag_multiplier
        slack = [0.0] * self.ncites     # slack -- sub-gradient

        # initial 'xLB' initial lower-bound should be obtained by relaxed model
        xLB = 1000.0    
        
        # initial 'xUB' initial UB sum all max travelCost
        for i in range(self.ncites):
            xUB += max(self.shipcost[i])

        # sentinel flag
        lbmodel = 0

        # build LR model
        self.build()
        # build fixed model
        self.build_fixed()
        # temporary linear expression
        obj_shipcost = quicksum(self.vship[i][j] * self.shipcost[i][j] \
                                      for i in range(self.ncites) \
                                      for j in range(self.ncites))

        '''
        main 'Lagrangian Relaxation' loop
        solve Lagrangian-relaxed problem 
        using Gradient-ascent to solve Lagrangian-dual problem
        ''' 
        for iter in range(self.iterlimit):
            # lagrangian relaxation term: sum u(j)(d(j) - sum x(i,j))
            obj_lagrangian = quicksum(xlambda[j] * (self.demand[j] - \
                                            quicksum(self.vship[i][j] for i in range(self.ncites))) \
                                            for j in range(self.ncites))
        
            self.mtrans.setObjective(obj_shipcost + obj_lagrangian, sense="minimize")
            
            # 'LB' model 求解松弛模型 -- obtain the lower bound
            self.mtrans.optimize()
            # =========================================================================================            
            # Calculate 'slack'=[sum x(i,j) - d(j)] -- 负sub-gradient  -- gradient ascent
            for j in range(self.ncites):
                slack[j] = sum(self.mtrans.getVal(self.vship[i][j]) for i in range(self.ncites)) - self.demand[j]
            
            # Find proper step size >>>
            # Update lower bound of obj if there is any improvement
            if self.mtrans.getObjVal() > xLB + 1e-6:
                xLB = self.mtrans.getObjVal()
                same = 0
            else:
                same += 1
            
            # update scale theta if theta does not change for a number of iterations(noChangeCntLimit)
            if same >= self.samelimit:
                scale /= 2.0
                same = 0
            
            # calculate 'l2-norm'
            norm = sum(slack[i]**2.0 for i in range(self.ncites))

            # update 'step-size'
            step = scale * (xUB - self.mtrans.getObjVal()) / norm

            # update Lagrangian multiplier
            for i in range(self.ncites):
                if xlambda[i] > step * slack[i]:
                    # slack 是负的梯度, 所以这里需要用减法
                    xlambda[i] -= step * slack[i]
                else:
                    xlambda[i] = 0.0
            
            # =========================================================================================            
            # get an upper bound of original model
            '''
            we relax the demand constraints, thus the relaxed model may select more facility so that the supply
            will ecxceed the demand
            '''
            # selected facility supply
            sumsbval = sum(self.supply[i] * self.mtrans.getVal(self.vbuild[i]) for i in range(self.ncites))
            sumdemand = sum(self.demand)
        
            if sumsbval - sumdemand >= 1e-6:
                # add relaxed constraints into LTP model and fix y, so that the model is easy to solve
                # this LTP model is same as the original model
                # but with fixed y
                # in lagrangian relaxation version, we relax these constraints
                # in this version, we add them back, aiming to obtain an UB
                lbmodel = 0
                
                # retrieve solution from LB model and fix it
                '''
                fix facility location variable and get an upper bound of original model
                fix the value of y (via revise the lb and ub)
                '''
                solution = []
                # retrieve solution from LB model and fix it
                for i in range(self.ncites):
                    solution.append(self.mtrans.getVal(self.vbuild[i]))
                    
                self.update_bound(solution)

                self.mtrans_ld.optimize()
                # update the UB
                temp = self.mtrans_ld.getObjVal()
                self.mtrans_ld.freeTransform()
                self.mtrans_ld.freeReoptSolve()
                xUB = min(xUB, temp)
                
                
            # update 'xLBlog', 'xUBlog', 'steplog', 'scalelog'
            self.xLBlog.append(xLB)
            self.xUBlog.append(xUB)
            self.steplog.append(step)
            self.scalelog.append(scale)
        
            # free the model        
            self.mtrans.freeTransform()
        
    def report(self):
        print("\n               *** Summary Report ***               \n")
        print("  Iter        LB               UB          scale        step")

        for i in range(len(self.xLBlog)):
            print("  %3d    %12.6f    %12.6f    %8.6f    %8.6f" \
                  % (i, self.xLBlog[i], self.xUBlog[i], self.scalelog[i], self.steplog[i]))

if __name__ == "__main__":
    loctrans = LocationTransport()
    loctrans.read("loctrans.dat")
    loctrans.solve()
    loctrans.report()


               *** Summary Report ***               

  Iter        LB               UB          scale        step
    0     1000.000000     2701.000000    1.000000    2.550519
    1     1000.000000     2598.000000    1.000000    1.436043
    2     1000.000000     2598.000000    0.500000    0.493849
    3     1000.000000     2598.000000    0.500000    1.053090
    4     1084.487699     2570.000000    0.500000    0.854127
    5     1129.855625     2570.000000    0.500000    0.895612
    6     1129.855625     2570.000000    0.500000    0.648179
    7     1304.290575     2570.000000    0.500000    0.947387
    8     1304.290575     2570.000000    0.500000    0.686879
    9     1304.290575     2403.000000    0.500000    1.564901
   10     1304.290575     2403.000000    0.250000    0.238752
   11     1304.290575     2403.000000    0.250000    0.505038
   12     1399.106623     2403.000000    0.250000    1.016086
   13     1399.106623     2348.000000    0.250000    0.601180
   14     1410.4